In [ ]:
import os
from tqdm import tqdm
from PIL import Image
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score


import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

In [ ]:
import random

seed_value = 42

random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed(seed_value)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 데이터셋 정의

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/train.csv')
val = pd.read_csv('/content/drive/MyDrive/val.csv')

In [ ]:
class clothdataset(object):
    def __init__(self, data_df, transform=None, test=False):
        self.data_df = data_df
        self.test = test
        self.transform = transform

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        img = Image.open(self.data_df.iloc[idx]['path']).convert('RGB')
        if self.transform:
            img = self.transform(img)
        if self.test:
            return img
        else:
            return img, self.data_df.iloc[idx]['class']

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [ ]:
train_dataset = clothdataset(train, transform=transform, test=False)
val_dataset = clothdataset(val, transform=transform, test=False)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 모델 정의

In [ ]:
# class Model(nn.Module):
#     def __init__(self):
#         super(Model, self).__init__()
#         self.layer1 = nn.Sequential(
#             nn.Conv2d(3, 64, 1, 1),
#             nn.ReLU(),
#         )
#         self.layer2 = nn.Sequential(
#         )
#         self.fc = nn.Linear(32*124*124, CLASSES)

#     def forward(self, x):
#         x = self.layer1(x)
#         x = x.view(-1, 32*124*124)
#         x = self.fc(x)
#         return x

In [ ]:
class VGG16(nn.Module):
    def __init__(self, CLASSES):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, CLASSES),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
def train(train_dataloader):
    model.to(device)

    for epoch in range(EPOCH):
        model.train()
        y_pred = []
        y_true = []
        running_loss = 0
        for x, y in tqdm(train_dataloader):
            y_true.extend(y)
            x = x.to(device)
            y = y.to(device)

            optimizer.zero_grad()

            output = model(x)
            _, tmp = torch.max(output, 1)
            y_pred.extend(tmp.cpu())
            loss = loss_fn(output, y)

            loss.backward()
            optimizer.step()

        running_loss += loss.item()
        print(f'Epoch: {epoch} | f1 score: {f1_score(y_pred, y_true, average="macro")} | loss: {running_loss / len(train_dataloader)}')

In [ ]:
EPOCH = 20
CLASSES = 12
device = "cuda" if torch.cuda.is_available() else "cpu"
model = VGG16(CLASSES)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/vgg_model.pth'))

In [ ]:
train(train_dataloader)

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/vgg_model.pth')

In [ ]:
def validation(val_dataloader):
    model.eval()
    y_pred = []
    y_true = []
    with torch.no_grad():
        for x, y in tqdm(val_dataloader):
            y_true.extend(y)
            x, y = x.to(device), y.to(device)
            output = model(x)
            _, pred = torch.max(output, dim=1)
            y_pred.extend(pred.cpu())
    print(f"Validation F1_score: {f1_score(y_pred, y_true, average='macro')}")

In [ ]:
validation(val_dataloader)